In [2]:
import pickle

In [3]:
# 행과열을 다루는 라이브러리
import pandas as pd
import numpy as np

In [9]:
# !pip install uvicorn
# !pip install fastapi

In [6]:
import uvicorn

In [12]:
from fastapi import FastAPI
# 인터페이스 데이터 관리를 위한 라이브러리
from pydantic import BaseModel

In [13]:
from fastapi.middleware.cors import CORSMiddleware
origins = ["*"]

app = FastAPI(title="ML API")

# CORS 미들웨어 추가
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # 모든 origin 허용
    allow_credentials=True,
    allow_methods=["GET", "POST", "PUT", "DELETE"],
    allow_headers=["*"],
)

In [18]:
# ★ 변경포인트 본인의 Feature로 변경
class InDataset(BaseModel):
    inHclus : int
    inPropercent : float
    inPromotion : str
    inHoliday : str

### 1. 학습결과 불러오기

In [14]:
with open("mlcore_skku.dump", "rb") as fr:
    loadedDump = pickle.load(fr)

In [15]:
features = loadedDump["feature"]
label=loadedDump["label"]
preProcessing = loadedDump["preprocessing"]
loadedModel = loadedDump["model"]

### 2. 예측 시뮬레이션

In [24]:
def hkfunc(inValue:float):
    """
    define: 함수다..이런이런..
    """
    # inValue = 5.7
    result = int(inValue)
    return result

In [ ]:
hkfunc(

In [25]:
@app.post("/predict", status_code=200)
async def predictSellout(x:InDataset):
    ### 화면 입력값
    inHclus = x.inHclus
    inPropercent = x.inPropercent
    inHoliday = x.inHoliday
    inPromotion = x.inPromotion
    ### 기존 훈련과 동일한 전처리과정 수행
    ynLabel = preProcessing[2]
    inHolidayConv = ynLabel.transform( [inHoliday] )[0]
    ynLabel = preProcessing[3]
    inPromotionConv = ynLabel.transform( [inPromotion] )[0]
    ### features 순서대로 데이터 생성 및 예측
    inDf = pd.DataFrame( [[  inHclus, inPropercent, inHolidayConv, inPromotionConv   ]])
    predictValue = loadedModel.predict(inDf)[0]
    result = { "prediction": predictValue }
    return result